# Introduction into Data Engineering

## Existing tools
Data bases
* MySQL
* PostegreSQL

Processing
* Apache Spark (uses DataFrame abstraction, language: PySpark, Py, Scala)
* Hadoop (structure queries for parallel processing)
    * HDFS
    * MapReduce
* Hive (runs on Hadoop MapReduce, is SQL)

Scheduling
* (Linux's `cron`)
* (Spotrify's Luigi)
* Apache AirFlow
* Oozie

In [ ]:
# EXAMPLE IN PYSPARK

# Print the type of athlete_events_spark
print(type(athlete_events_spark))

# Print the schema of athlete_events_spark
print(athlete_events_spark.printSchema())

# Group by the Year, and find the mean Age
print(athlete_events_spark.groupBy('Year').mean('Age'))

# Group by the Year, and find the mean Age
print(athlete_events_spark.groupBy('Year').mean('Age').show())

## Processing
* Clean data
* Aggregate data
* Join data

Data processing must be distributed over clusters of virtual machines.

In [ ]:
from multiprocessing import Pool

multiprocessing.Pool

def take_mean_age(year_and_group):
    year, group = year_and_group
    return pd.DataFrame({"Age": group['Age'].mean()}, index = [year])

with Pool(4) as p:
    results = p.map(take_mean_age, athlete_events.groupby('Year'))
    
results_df = pd.concat(results)

In [ ]:
import dask.dataframe as dd

# Partition datafrom into 4
event_dask = dd.from_pandas(athlete_evetns, npartitions = 4)

# Run parallel compuations on each partition
results_df = event_dask.groupby('Year').Age.mean().compute()

## Scheduling
* Plan jobs with specific time intervals
* Resolve dependency requirements of jobs

Tasks to make sure:
- Jobs run in a specific order and all dependencies are resolved correctly
- Jobs run at a specific time each day

For scheduling events, you can use Direct Acyclic Graph (DAGs):
* Set of nodes
* Directed edges
* No cycles

Recommanded tool: Apache Airflow
* Created at Airbnb
* Uses DAGs
* Uses Python

In [ ]:
from airflow.models import DAG
from airflow.operators.python_operators import PythonOperator

# Create a DAG
dag = DAG(dag_id = 'example_dag', ..., scheduling_interval = '0 * * * *') # Every hour at minute 0 
# minute [0-59], hour [0-23], monthday [1-31], month [1-12], weekday [0-6]

# Define operations
start_cluster = StartClusterOperator(task_id = 'start_cluster')
ingest_customer_data = SparkJobOperator(task_id = 'ingest_customer_data' , dag = dag)
ingest_product_data = SparkJobOperator(task_id = 'ingest_product_data' , dag = dag)
enrich_customer_data = PythonOperator(task_id = 'enrich_customer_data' , dag = dag) 
    # python_callable = <function ETL process>
    # op_kwargs = {"argument": argument_var}

# Set up dependency flow [set_upstream. set_downstream]
start_cluster.set_downstram(ingest_customer_data)
ingest_customer_data.set_downsteam(enrich_customer_data)
ingest_product_data.set_downsteam(enrich_customer_data)

# Save the file.py in ~/airflow/dags/

### Storage locaties

Data should be stored at different locations, for hackers and destruction of the datacenter.

* Azure
* Google cloud
* Amazon web services

Storage
- Azure: Azure Blob Storage
- Google: Google Cloud Storage
- Amazon: AWS3

Compute 
- Azure: Azure Virual Machines
- Google: Google Compute Engine
- Amazon: SWS EC2

Databases
- Azure: Azure SQL database
- Google: Google Cloud SQL
- Amazon: AWS RDS  